https://x.com/krismicinski/status/1856341897331003842  "Trying to turn in my class material into a few recorded lectures on CDCL. I have seen some good talks on CDCL history by experts (e.g., Armin Biere's Simmons institute talk), but think there is still room for more discussion that starts from a as-functional-as-possible style."

2 watched literal is related to simplex tableau method?
https://types.pl/@krismicinski/113470474347084554

Jules Jacobs - Retconning out DPLL and treat it as iterative clause learning from the start?

In [ ]:
# two watched literal
Lit = int
clauses = []



# SAT

https://en.wikipedia.org/wiki/DPLL_algorithm
https://chatgpt.com/c/66e9eb94-8984-8008-9db9-c0190f98e25c

In [ ]:
from typing import NamedTuple

Literal = NamedTuple('Literal', [('idx', int), ('polarity', bool)])
Clause = NamedTuple('Clause', [('literals', list)])
Formula = NamedTuple('Formula', [('clauses', list[Clause]), ('num_vars', int)])

Assignment = NamedTuple('Assignment', [('values', list[bool])])
Pasn = NamedTuple('Pasn', [('assignment', Assignment), ('satisfiable', bool)])


Can we use z3 to do cnf transformation?
https://microsoft.github.io/z3guide/docs/strategies/summary#tactic-tseitin-cnf

def dimacs(self, include_names=True): also apparently solver has a dimacs printer...

https://stackoverflow.com/questions/13059096/z3-and-dimacs-output
https://github.com/RobinDavid/pydimacs

I put it into knuckeldragger


In [32]:
from z3 import *

x,y,z,w = Bools("x y z w")
G = Goal()
G.add(Implies(x, And(z,w)))
G.add(Implies(y, Or(x,z)))
t1 = Tactic('tseitin-cnf')
tactic = z3.Then('simplify', 'bit-blast', 'tseitin-cnf')
for c in tactic(G):
    print(c)
    print(c.dimacs())

import subprocess
class SATSolver():
    def __init__(self):
        self.G = z3.Goal()
    def add(self, clause):
        self.G.add(clause)
    def check(self, debug=False):
        t = z3.Then('simplify', 'bit-blast', 'tseitin-cnf')
        c = t(self.G)
        assert(len(c) == 1)
        with open("/tmp/sat.cnf", "w") as f:
            f.write(c[0].dimacs())
            f.flush()
        self.res = subprocess.run(["kissat", "--relaxed", "/tmp/sat.cnf"], capture_output=True)
        res = self.res.stdout.decode()
        if "s SATISFIABLE" in res:
            return z3.sat
        elif "s UNSATISFIABLE" in res:
            return z3.unsat
        else:
            return z3.unknown

s = SATSolver()
s.add(Implies(x, And(z,w)))
s.add(Implies(y, Or(x,z)))
assert s.check() == z3.sat
#print(s.res.stdout.decode())



[Or(z, Not(x)), Or(w, Not(x)), Or(x, z, Not(y))]
p cnf 4 3
1 -2 0
3 -2 0
2 1 -4 0
c 2 x
c 4 y
c 1 z
c 3 w


In [26]:
! cat /tmp/sat.cnf

p cnf 12 29
52 -48 -49 0
52 -48 0
52 -49 0
48 49 -52 0
52 50 46 -53 0
50 -53 -52 -46 0
52 -50 -53 -46 0
46 -50 -53 -52 0
-50 -52 53 -46 0
52 46 -50 53 0
50 46 -52 53 0
52 50 53 -46 0
-53 0
48 -54 -44 0
44 -48 -54 0
-48 54 -44 0
44 48 54 0
-54 0
50 55 0
52 55 0
-50 -52 -55 0
47 51 55 0
47 -51 -55 0
51 -47 -55 0
-51 -47 55 0
45 48 49 0
45 -48 -49 0
-48 49 -45 0
48 -49 -45 0

In [34]:
x = BitVec('x', 4)
y = BitVec('y', 4)
s = SATSolver()
s.add(x == y + 3)
assert s.check() == z3.sat
#print(s.res.stdout.decode())

s = SATSolver()
s.add(x == x + y)
s.add(y != z3.BitVecVal(0, 4))
assert s.check() == z3.unsat


In [15]:
from z3 import *
a,b,c = Ints("a b c")
s = SimpleSolver()
s.add(a == b)
s.add(b == c)
s.check()
print([s.root(x) for x in [a,b,c]])
[s.next(x) for x in [a,b,c]]    
#s.next(b)

[b, b, b]


[b, c, a]

In [ ]:
from z3 import *
s = SimpleSolver()
x, y, z = Ints('x y z')
s.add(x == y)
s.add(y == z)
s.check()
print(s.root(x), s.root(y), s.root(z))
print(s.next(x), s.next(y), s.next(z))

In [2]:
from z3 import *

x,y,z,w = Bools("x y z w")
G = Goal()
G.add(Implies(x, And(z,w)))
t1 = Tactic('tseitin-cnf')
t1(G)


[[Or(z, Not(x)), Or(w, Not(x))]]

Match the z3 api 

In [ ]:
from dataclass import dataclass

Id = int

@dataclass
class Context():
    hashcons : dict[tuple[Id] | str | int, Id]
    def intern(self, x : tuple[Id] | str | int) -> Id:
        if x in self.hashcons:
            return self.hashcons[x]
        else:
            id = len(self.hashcons)
            self.hashcons[x] = id
            return id

  


@dataclass(frozen=True)
class AstRef():
    ctx : Context
    ast : Id
    def __eq__(self, other : AstRef) -> bool:
        assert self.cxt == other.ctx
        return self.ast == other.ast
    def __hash__(self) -> int:
        return hash(self.ast)
    
    # fill in
_main_ctx : Context = Context({})
def get_ctx(ctx):
    if ctx is None:
        return _main_ctx
    return ctx

def to_symbol(x : int | str, ctx=None) -> int:
    ctx = get_ctx(ctx)
    return ctx.intern(x)
    
def is_ast(x):
    return isinstance(x, AstRef)

@dataclass(frozen=True)
class SortRef(AstRef):

@dataclass(frozen=True)
class ExprRef(AstRef):
    # fill in
    ast : AstRef

With my egraph, if i wrote a sat solver, I'd have a mini
Put in SAT comp?

[The silent revolution of sat](https://news.ycombinator.com/item?id=36079115#36081904)

[creusat](https://sarsko.github.io/_pages/SarekSkot%C3%A5m_thesis.pdf) <https://github.com/sarsko/CreuSAT>

https://fmv.jku.at/fleury/papers/Fleury-thesis.pdf

https://github.com/marijnheule/microsat

https://github.com/msoos/minisat-v1.14

https://x.com/SoosMate/status/1760817528236818456?s=20 phind know sat solvers
"Yes (about the role of Kitten in Kissat) but the explanation is still bogus. It sounds like a weak student who does not understand anything but tries to pass the exam by randomly arguing about concepts discussed in this context. I probably would let it pass (but just)."

https://sahandsaba.com/understanding-sat-by-implementing-a-simple-sat-solver-in-python.html
wait this blog is sweet

https://easychair.org/publications/open/b7Cr CDCL as rewrite syste,


In [ ]:
# parsing DIMACS
def parse_dimacs(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    clauses = []
    for line in lines:
        if line[0] == 'c':
            continue
        if line[0] == 'p':
            n_vars = int(line.split()[2])
            continue
        clause = [int(x) for x in line.split()[:-1]]
        clauses.append(clause)
    return n_vars, clauses

In [ ]:
import numpy
nclauses = 10
clause_size = 
np.zeros((len(clauses), ) , dtype=np.uint8)



In [ ]:
def unit_prop(solution, clauses):
    while True:
        unit_clauses = [c for c in clauses if len(c) == 1]
        if len(unit_clauses) == 0:
            break
        unit_clause = unit_clauses[0]
        unit_literal = unit_clause[0]
        solution[abs(unit_literal)] = 1 if unit_literal > 0 else 0
        clauses = [c for c in clauses if unit_literal not in c]
    return solution, clauses





In [ ]:
def CDCL():
    while True:
        if [] in clauses: return UNSAT
        elif in_conflict(): learn(); backtrack()
        elif not free_vars: return SAT
        elif should_propagate(): propagate()
        elif should_simplify(): simplify()
        elif should_restarty(): restart()
        elif: should_prune(): prune*()
        else:
            var= chooce_var()
            sign = choose_sign()
            assign(var,sign)


Metatarki
https://z3prover.github.io/papers/z3internals.html

https://github.com/sarsko/CreuSAT/blob/master/Friday/src/lib.rs Friday

In [ ]:
assignment= list[bool]
lit = tuple[int, bool]
clause = list[lit]


def eval(formula, assignment):
    pass

def solve(formula):
    assignment = [[None] * f.num_vars]
    ix = 0
    while True:
        if ix == f.num_vars:


# 
assignment = 0

while True:
    assignment += 1
    eval(formula, assignment)



# SMT
difference logic
https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csgraph.shortest_path.html


Engineering DPLL(T) + sautriation
https://leodemoura.github.io/files/z3spc.pdf

https://homepage.cs.uiowa.edu/~tinelli/papers/NieOT-JACM-06.pdf Solving SAT and SAT Modulo Theories: from an
Abstract Davis-Putnam-Logemann-Loveland
Procedure to DPLL(T)

https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=45036ae8e05d71ea39253f863cfa270b6cdd9aa2 simplify paper



In [6]:
from typing import NamedTuple


ctx = {}

#@dataclass
class AstRef(NamedTuple):
    #head : str
    #sort : AstRef
    #children : tuple[AstRef, ...]
    def eq(self, other):
        return self == other
    def sexpr(self):
        return 



class SortRef(AstRef):
    name : str

# smtlib3
class TVarRef(SortRef):
    pass

class TForAll(SortRef):
    pass

    
class FuncDeclRef(AstRef):
    name : str
    dom : tuple[SortRef, ...]
    cod : SortRef
    def range(self):
        return self.cod
    def arity(self):
        return len(self.dom)
    def __call__(self, *args):
        return ExprRef(self, args)   

class ExprRef(AstRef):
    _decl : FuncDeclRef
    _args : tuple[ExprRef, ...]
    def decl(self):
        return self._decl
    def arg(self, i):
        return self._args[i]
    def children(self):
        return self._args
    def sort(self):
        return self._decl.cod


class BoolSortRef(SortRef):
    name = "Bool"

def BoolSort():
    return BoolSortRef()

class VarRef(AstRef):
    ind : int
#def QuantifierRef(ExprRef):


#isinstance(BoolSort(),NamedTuple)

def is_app(x):
    return isinstance(x, ExprRef)
def is_var(x): return isinstance(x, VarRef)
def Function(name, *args):
    return FuncDeclRef(name, args[:-1], args[-1])

def And(*args):
    AndDecl = Function("And", *args)
    #return AndDecl(*args)


TypeError: isinstance() arg 2 must be a type, a tuple of types, or a union

In [9]:
import z3
x,y = z3.Bools("x y")
z3.ForAll([x,y], z3.And(x,y)).decl()

Z3Exception: Z3 application expected

In [34]:
%%file /tmp/test.lean

inductive Expr : Type
| UVar : String -> Expr
| App : String -> List Expr -> Expr

deriving instance BEq, Hashable, Inhabited, Repr for Expr

def main : IO Unit :=
    do
        IO.println "Hello, world!"
        IO.println "hi"
        IO.println (repr (Expr.App "c" []))

Overwriting /tmp/test.lean


In [35]:
!lean --run /tmp/test.lean

Hello, world!
hi
Expr.App "c" []
